# Simulating plus EV strategies
This notebook is contigent on running UF_Pinnacle_PlusEV.ipynb, which scrapes/computes/stores recommended Underdog Fantasy bets.

In [ ]:
import random
import numpy as np
import plotly.graph_objects as go

import plotly.express as px

import pandas as pd

import matplotlib.pyplot as plt

rng = np.random.default_rng()

In [ ]:
recommended_bets = pd.read_csv('recommended_bets.csv')
unique_combo_df = pd.read_csv('recommended_bets_unique.csv')
value_df = pd.read_csv('value_df.csv')

In [ ]:
payout_multipliers = {
    3: {'standard': 6, 'insured': 3},
    4: {'standard': 10, 'insured': 6},
    5: {'standard': 20, 'insured': 10},
}

In [ ]:
value_df

,Unnamed: 0,player,game,time,event,type,line_uf,line_pinn,prob,o_u
0,0,Kawhi Leonard,LAC @ MEM,- 7:00PM CST,3-Pointers Made,both,2.0,1.5,0.649123,over
1,1,Paul George,LAC @ MEM,- 7:00PM CST,3-Pointers Made,both,3.0,3.5,0.638989,under
2,2,Aaron Nesmith,IND @ ATL,- 6:30PM CST,Pts + Rebs + Asts,both,17.0,17.5,0.609375,under
3,3,Kawhi Leonard,LAC @ MEM,- 7:00PM CST,Rebounds,both,6.0,6.5,0.609375,under
4,4,Alperen Sengun,HOU @ DET,- 6:30PM CST,Assists,both,5.0,4.5,0.598394,over
...,...,...,...,...,...,...,...,...,...,...
105,105,Alperen Sengun,HOU @ DET,- 6:30PM CST,Assists,both,5.0,4.5,0.469484,under
106,106,Kawhi Leonard,LAC @ MEM,- 7:00PM CST,Rebounds,both,6.0,6.5,0.462963,over
107,107,Aaron Nesmith,IND @ ATL,- 6:30PM CST,Pts + Rebs + Asts,both,17.0,17.5,0.458716,over
108,108,Paul George,LAC @ MEM,- 7:00PM CST,3-Pointers Made,both,3.0,3.5,0.434783,over


In [ ]:
recommended_bets

,Unnamed: 0,combo,EV,num_picks,bet_type,total_prob
0,0,"(5, 8, 10)",0.429874,3,standard,0.204268
1,17710,"(5, 8, 10, 13, 14)",0.420077,5,standard,0.067623
2,17711,"(5, 8, 10, 13, 15)",0.415582,5,standard,0.067409
3,1,"(5, 8, 13)",0.412563,3,standard,0.201795
4,17728,"(5, 8, 10, 14, 15)",0.411152,5,standard,0.067198
...,...,...,...,...,...,...
70373,70375,"(30, 31, 33, 34, 36)",-0.434359,5,insured,0.051422
70374,70366,"(29, 30, 33, 34, 36)",-0.434359,5,insured,0.051422
70375,70371,"(29, 32, 33, 34, 36)",-0.434359,5,insured,0.051422
70376,70370,"(29, 31, 33, 34, 36)",-0.434359,5,insured,0.051422


In [ ]:
payout_multipliers = {
    3: {'standard': 6, 'insured': 3},
    4: {'standard': 10, 'insured': 6},
    5: {'standard': 20, 'insured': 10},
}


### Visualizing long-term simulated bankroll from +EV Bets

In [ ]:
def generate_simulations(combo_prob, wager, payout_multiplier, num_simulations=1000, num_bets=100):
    """
    Generate multiple simulations of a game.

    :param combo_prob (float): The probability of winning the bet.
    :param wager (float): The amount wagered per bet.
    :param payout_multiplier (int): The payout multiplier for a winning bet.
    :param num_simulations (int): The number of simulations to run.
    :param num_runs (int): The number of times to run the simulation.
    :return: A matrix of simulations.
    """
    # Create a matrix to store the results of each simulation
    simulations = np.zeros((num_simulations, num_bets))
    
    for i in range(num_simulations):
        profit = 0
        bankroll = []
        
        for _ in range(num_bets):
            prob = np.random.random()
            if prob <= combo_prob:
                profit += wager * payout_multiplier
            else:
                profit -= wager
            bankroll.append(profit)
        
        simulations[i, :] = bankroll
        
    return simulations

def plot_simulation(value_df, combo, simulations_matrix):
    """
    This visualizes the simulation results of multiple bet combinations and displays the individual simulation results and the average earnings over time.


    :param value_df (DataFrame): DataFrame containing bet details and probabilities.
    :param combo (list): Indices representing a specific bet combination.
    :param simulations_matrix: A matrix of simulations (returned by generate_simulations()).
    """
    bets_df = value_df.iloc[list(combo)]
    comp_prob = bets_df['prob'].prod()
    display(bets_df)
    
    fig = px.line(title=f"Avg Profit [{len(simulations_matrix[0])} bets x {len(simulations_matrix)} sims]",
                  line_shape="spline",)
    
    # plot each simulation
    for i in range(simulations_matrix.shape[0]):
        fig.add_scatter(x=list(range(simulations_matrix.shape[1])), y=simulations_matrix[i, :],
                        mode='lines', line=dict(width=1, color='grey'), line_shape="spline",
                        name=f'sim {i}',
                        opacity=0.1,
                        showlegend=False)
    
    # plot the average of all simulations
    avg_bankroll = np.mean(simulations_matrix, axis=0)
    fig.add_scatter(x=list(range(len(avg_bankroll))), y=avg_bankroll, mode='lines', line=dict(width=3, color='blue'), name='Avg. Earnings')
    
    fig.update_xaxes(title_text='Number of Bets')
    fig.update_yaxes(title_text='Bankroll')
    
    annotation_text = ""
    for idx, row in bets_df.iterrows():
        annotation_text += f"{row['game']} | {row['player']} | {row['event']} | {row['o_u']} <br>"
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=0.5,
        y=1.3,
        text=annotation_text,
        showarrow=False,
    )
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=1,
        y=1.2,
        text=f'Compound Probability {comp_prob:.4f}',
        showarrow=False,
    )
    
    fig.show()

In [ ]:
# Example usage
combo = (5, 8, 10)
combo_prob = value_df.iloc[list(combo)]['prob'].prod()

simulations_matrix = generate_simulations(combo_prob=combo_prob, wager=10, payout_multiplier=6,
                                          num_simulations=100, num_bets=50)

plot_simulation(value_df=value_df,
                combo=combo,
                simulations_matrix=simulations_matrix)

,Unnamed: 0,player,game,time,event,type,line_uf,line_pinn,prob,o_u
5,5,Jabari Smith Jr.,HOU @ DET,- 6:30PM CST,Pts + Rebs + Asts,both,26.5,26.5,0.596774,under
8,8,James Harden,LAC @ MEM,- 7:00PM CST,Assists,both,9.5,9.5,0.586777,under
10,10,Fred VanVleet,HOU @ DET,- 6:30PM CST,Points,both,19.5,19.5,0.583333,under


In [ ]:
recommended_bets.query('combo=="(5, 8, 10)"')

,Unnamed: 0,combo,EV,num_picks,bet_type,total_prob
0,0,"(5, 8, 10)",0.429874,3,standard,0.204268
35192,1540,"(5, 8, 10)",-0.182929,3,insured,0.204268


In [ ]:
# TODO add documentation and event names to plot rather than indices
def compare_sims(value_df, combos, wager=10, payout_multiplier=6, num_simulations=100, num_bets=50):
    profit_dict = {}
    combo_info = {}
    for combo in combos:
        profit_dict[str(combo)] = {}
        info_text = ""
        bets_df = value_df.iloc[list(combo)]
        
        for idx, row in bets_df.iterrows():
            info_text += f"{row['game']} | {row['player']} | {row['event']} | {row['o_u']} <br>"
        combo_info[str(combo)] = info_text
            
        combo_prob = bets_df['prob'].prod()
        sim_matrix = generate_simulations(combo_prob, wager, payout_multiplier, num_simulations, num_bets)
        avg_profit = np.mean(sim_matrix, axis=0)
        profit_dict[str(combo)] = avg_profit

    profits_df = pd.DataFrame(profit_dict)

    fig = px.line(title=f"Avg Profit [{num_bets} bets x {num_simulations} sims] for {len(combos)} plays")
    
    # Add each combo as a separate trace
    for combo in profits_df.columns:
        fig.add_scatter(x=profits_df.index, y=profits_df[combo], name=combo)
        fig.data[-1].hovertemplate = combo_info[combo] + '<br>Avg. Bankroll: %{y}<extra></extra>'

    
    fig.update_xaxes(title_text='Number of Bets')
    fig.update_yaxes(title_text='Bankroll')
    fig.update_layout(hovermode="x")
    print(combo_info)
    fig.show()
            
compare_sims(value_df = value_df, 
             combos = [(5,8,9), (6,2,1), (3,4,5)],
             wager = 10,
             payout_multipliers=6,
             num_simulations=100,
             num_bets=50)
        

{'(5, 8, 9)': 'HOU @ DET | Jabari Smith Jr. | Pts + Rebs + Asts | under <br>LAC @ MEM | James Harden | Assists | under <br>HOU @ DET | Jabari Smith Jr. | Points | under <br>', '(6, 2, 1)': 'HOU @ DET | Jabari Smith Jr. | Rebounds | over <br>IND @ ATL | Aaron Nesmith | Pts + Rebs + Asts | under <br>LAC @ MEM | Paul George | 3-Pointers Made | under <br>', '(3, 4, 5)': 'LAC @ MEM | Kawhi Leonard | Rebounds | under <br>HOU @ DET | Alperen Sengun | Assists | over <br>HOU @ DET | Jabari Smith Jr. | Pts + Rebs + Asts | under <br>'}
